In [3]:
import pandas as pd
import pandapower as pp
import numpy as np
import pandapower.networks as pn

net = pn.GBreducednetwork() #importing the database as the network for pandapower

net.load.scaling = 1.012
net.sgen.scaling = 1.012
net.gen.scaling = 1.012

results=pd.DataFrame() #creating a new dataframe called "results" for selecting the critical lines
results["n_overload"]="" #adding a new column in the "results" for the number of overloaded lines
results["excess_overload"]="" #adding a new column in the "results" for the total overload excess
results["overload_per_cable"]="" 

pp.runpp(net) #running the powerflow analysis with all lines intact
pp.to_excel(net,"GBreducednetworks.xlsx") #exporting the results into an excel document

results.at[:,"loading percent"]=net.res_line["loading_percent"] #adding the loading percent for each line into the "results"
x = range(0, len(net.res_line.index)) #setting a loop for dropping every line one at a time
for i in x:
    
    pp.drop_lines(net, [i]) #dropping a line
    
    pp.runpp(net)  #running the powerflow analysis with the line dropped
        
    net.res_line.loc[net.res_line.loading_percent < 100, 'equal_or_greater_than_100'] = 0 
    net.res_line.loc[net.res_line.loading_percent >= 100, 'equal_or_greater_than_100'] = 1 
    #creating a new column in the net.res_lines database for the number of overloaded lines
    #setting the value equal to 0 if the line is not overloaded and 1 if it is
        
    net.res_line.loc[net.res_line.loading_percent > 100, 'excess_over_100'] = (net.res_line.loading_percent - 100)
    net.res_line.loc[net.res_line.loading_percent <= 100, 'excess_over_100'] = 0
    #creating a new column in the net.res_lines database for the excess overload
    #setting the value equal to the excess overload (if the line is overloaded) and 0 if it is not
        
    n_overload = np.sum(net.res_line.equal_or_greater_than_100)
    #creating a new variable to equal to the total number of overloaded lines as a result of dropping a given line
    
    excess_overload = np.sum(net.res_line.excess_over_100)
    #creating a new variable to equal to the sum of excess overloads on the lines as a result of dropping a given line
    
    results.at[i,"n_overload"]=n_overload #adding the number of overloaded lines to the "results"
    results.at[i,"excess_overload"]= excess_overload #adding the total excess overload on the lines to the "results"
    results.at[i,"overload_per_cable"]= excess_overload/n_overload
    
    net = pn.GBreducednetwork() #resetting the network (putting back the droppped line)

print(results) #displaying the results
sort_by_size = results.sort_values('overload_per_cable')
print(sort_by_size.tail(n=10))
sort_by_size = results.sort_values('loading percent')
print(sort_by_size.tail(n=10))
sort_by_size = results.sort_values('excess_overload')
print(sort_by_size.tail(n=10))
sort_by_size = results.sort_values('n_overload')
print(sort_by_size.tail(n=10))
print("Program finished successfully") #signalling the end of the program

   n_overload excess_overload overload_per_cable  loading percent
0          12         294.933            24.5777        36.181384
1          12         278.121            23.1768        37.198541
10         12           278.4               23.2         4.991055
11         12           278.4               23.2         4.991055
12         12         276.711            23.0593        47.425998
..        ...             ...                ...              ...
82         11         308.076            28.0069        84.395014
83         11         308.076            28.0069        84.395014
84         12          296.11            24.6758        24.497395
85         12          296.11            24.6758        24.497395
9          13         314.383            24.1833        60.688789

[86 rows x 4 columns]
   n_overload excess_overload overload_per_cable  loading percent
82         11         308.076            28.0069        84.395014
83         11         308.076            28.0069     